In [1]:
import numpy as np
from tqdm import tqdm
from ecit import *

/opt/anaconda3/envs/graph/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
np.random.seed(1)
random.seed(1)

t = 1000
n_list = [800, 1200, 1600]
ensCIT = [(cmiknn, 1, p_alpha2)]
zDiss = ['gaussian'] 
noiseDiss = ['t4','t3','t2','t1'] 

results_table = []

for n in n_list:
    for zDis in zDiss:
        for cit, k, p_ensemble in ensCIT:
            row = [n, zDis.capitalize(), cit.__name__ + str(k) + p_ensemble.__name__]

            for noiseDis in tqdm(noiseDiss, desc=f"{n:>5}, {zDis:>8}, {k:>3},{p_ensemble.__name__:>11}"):
                eI = 0
                eII = 0
                for i in range(t):
                    retries = 0
                    while retries < 5:
                        try:
                            dataI = np.hstack((generate_samples(n=n,indp='C',z_dis=zDis, noise_dis=noiseDis,noise_std=1)))
                            dataII = np.hstack((generate_samples(n=n,indp='N',z_dis=zDis, noise_dis=noiseDis,noise_std=1)))
                            obj_ECIT = ECIT(dataI, cit, p_ensemble, k if k<100 else int(n/k))
                            pI = obj_ECIT([0], [1], [2])
                            obj_ECIT = ECIT(dataII, cit, p_ensemble, k if k<100 else int(n/k))
                            pII = obj_ECIT([0], [1], [2])
                            break
                        except Exception as e:
                            retries += 1
                            print(f"Retry times {retries}: {e}")
                            if retries > 5: raise e
                    if pI<0.05:
                        eI += 1
                    if pII>0.05:
                        eII += 1
                eI = eI/t
                power = (t - eII)/t
                row.append(eI)
                row.append(power)
            results_table.append(row)

results_table

 1600, gaussian,   1,   p_alpha2: 100%|██████████| 4/4 [37:23:21<00:00, 33650.41s/it]   


[[800,
  'Gaussian',
  'cmiknn1p_alpha2',
  0.1,
  0.977,
  0.106,
  0.962,
  0.123,
  0.948,
  0.095,
  0.942],
 [1200,
  'Gaussian',
  'cmiknn1p_alpha2',
  0.101,
  0.988,
  0.106,
  0.985,
  0.095,
  0.965,
  0.118,
  0.966],
 [1600,
  'Gaussian',
  'cmiknn1p_alpha2',
  0.11,
  0.992,
  0.096,
  0.995,
  0.109,
  0.984,
  0.098,
  0.967]]

In [3]:
data = results_table

data = [
    [round(item, 3) if isinstance(item, float) else item for item in sublist]
    for sublist in data
]
for row in data:
    print("{:<6} {:<12} {:<18} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f} {:<6.3f}".format(*row))

800    Gaussian     cmiknn1p_alpha2    0.100  0.977  0.106  0.962  0.123  0.948 
1200   Gaussian     cmiknn1p_alpha2    0.101  0.988  0.106  0.985  0.095  0.965 
1600   Gaussian     cmiknn1p_alpha2    0.110  0.992  0.096  0.995  0.109  0.984 
